# Query for Failed Files of an Import Job Using the Arize GraphQL API

Your import jobs contain files which Arize attempts to read and upload into the platform. Sometimes, a file may not be able to be successfully uploaded for some reason. One such reason could be an issue with the formatting of your data in a particular row or column of the file. In order to make sure that all your files are successfully uploaded, you may wish to query an import job that you've created for any failed files. If they exist, the file path and a helpful error message are provided for you, and you can rememdy the issue by fixing the error and rename the file to trigger a re-read.

### Step 1: Initialize the GraphQL Client using your developer API key

In [ ]:
!pip install -q gql[all]
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

### Get your API key
First, make sure you have developer permissions. If you are able to visit the [API explorer](https://app.arize.com/graphql), then you have developer permissions. If not, please ask your Account Admin to provide you with access. 

The API key can be retrieved from the [API explorer](https://app.arize.com/graphql) page. Click the button on the top right called "Get Your API Key." A modal will pop up with your key, copy that into the `API_KEY` constant below. 

NOTE: this key is different than the SDK key used to send data to Arize.

In [ ]:
API_KEY = "API_KEY"

# Select your transport with a defined URL endpoint
transport = RequestsHTTPTransport(
    url="https://app.arize.com/graphql/", headers={"x-api-key": API_KEY}
)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

### Step 2: Execute a GraphQL query to get all your import jobs

In [ ]:
# We start this query from your space. Spaces have globally unique IDs. You can get your space ID by visiting app.arize.com.
# The url will be in this format: https://app.arize.com/organizations/:orgId/spaces/:spaceId
# NOTE: this is not the same as the space key used to send data using the SDK
SPACE_ID = "SPACE_ID"


# A re-usable query for fetching your import jobs, a page at a time
import_jobs_query = gql(
    """
    query getImportJobs($spaceId: ID!, $cursor: String) {
        space: node(id: $spaceId) {
            ... on Space {
                name
                importJobs (first: 50, after: $cursor) {
                    pageInfo {
                        endCursor
                    }
                    edges {
                        importJob: node {
                            id
                            modelName
                        }
                    }
                }
            }
        }
    }
"""
)

# Base query parameters for fetching import jobs
params = {"spaceId": SPACE_ID}
# An array of models that we will append to
import_jobs = []
space_name = ""


# Execute the query on the transport. Continue to pull data until there are no more import jobs
while True:
    paged_response = client.execute(import_jobs_query, params)
    space_name = paged_response["space"]["name"]
    # Append the import jobs to your list
    import_jobs.extend(paged_response["space"]["importJobs"]["edges"])
    # If there is another page of information, point the cursor to the next page and fetch more
    end_cursor = paged_response["space"]["importJobs"]["pageInfo"]["endCursor"]
    print("pageInfo end_cursor %s" % (end_cursor))
    if end_cursor:
        print("There is another page of import jobs. Loading more.")
        params["cursor"] = end_cursor
    else:
        print("No more import jobs to pull. The list is complete!")
        break

print("Retrieved {} import jobs".format(len(import_jobs)))

#### Step 2a: Print out some of the import jobs to check that exports are expected

In [ ]:
import pandas as pd

# The import jobs have a nested JSON structure, let's flatten it into a data frame
import_jobs_df = pd.json_normalize(import_jobs, sep=".")
import_jobs_df.head()

### Step 3: Save import jobs in a spreadsheet to reference for jobIds

In [ ]:
!pip install gspread
# src: https://colab.research.google.com/notebooks/snippets/sheets.ipynb
from google.colab import auth

auth.authenticate_user()

import gspread
from google.auth import default

creds, _ = default()

gc = gspread.authorize(creds)

sheet_name = f"{space_name} import jobs"
sh = gc.create(sheet_name)

# Open our new sheet and add some data.
worksheet = gc.open(sheet_name).sheet1

# Let's now write the dataframe to google sheets
worksheet.update(
    [import_jobs_df.columns.values.tolist()] + import_jobs_df.values.tolist()
)

# print the URL
print(sh.url)

### Step 4: Query for failed files on a particular import job

In [ ]:
# We start this query from your job. Jobs have globally unique IDs which were queried for in Step 2 above.
JOB_ID = "YOUR_IMPORT_JOB_ID"


# A re-usable query for fetching your files, a page at a time
# The status passed into the files connection can be one of FAILED, PENDING, COMPLETE, SKIPPED, or CANCELED. If no status paramter is provided, all files are returned.
failed_files_query = gql(
    """
    query getFailedFiles($jobId: ID!, $cursor: String) {
        fileImportJob: node(id: $jobId) {
            ... on FileImportJob {
                modelName
                files (first: 50, status: FAILED, after: $cursor) {
                    pageInfo {
                        endCursor
                    }
                    edges {
                        file: node {
                            id
                            filePath
                            error {
                              message
                            }
                        }
                    }
                }
            }
        }
    }
"""
)

# Base query parameters for fetching import jobs
params = {"jobId": JOB_ID}
# An array of models that we will append to
files = []
model_name = ""


# Execute the query on the transport. Continue to pull data until there are no more import jobs
while True:
    paged_response = client.execute(failed_files_query, params)
    space_name = paged_response["fileImportJob"]["modelName"]
    # Append the import jobs to your list
    files.extend(paged_response["fileImportJob"]["files"]["edges"])
    # If there is another page of information, point the cursor to the next page and fetch more
    end_cursor = paged_response["fileImportJob"]["files"]["pageInfo"][
        "endCursor"
    ]
    print("pageInfo end_cursor %s" % (end_cursor))
    if end_cursor:
        print("There is another page of files. Loading more.")
        params["cursor"] = end_cursor
    else:
        print("No more files to pull. The list is complete!")
        break

print("Retrieved {} files".format(len(files)))

#### Step 4a: Print our some of the files to check that exports are expected

In [ ]:
import pandas as pd

# The import jobs have a nested JSON structure, let's flatten it into a data frame
failed_files_df = pd.json_normalize(files, sep=".")
failed_files_df.head()

### Step 5: Save files and errors in a spreadsheet for reference in order to fix your errors

In [ ]:
sheet_name = f"{model_name} files"
sh = gc.create(sheet_name)

# Open our new sheet and add some data.
worksheet = gc.open(sheet_name).sheet1

# Let's now write the dataframe to google sheets
worksheet.update(
    [failed_files_df.columns.values.tolist()] + failed_files_df.values.tolist()
)

# print the URL
print(sh.url)